In [25]:
import os


In [2]:
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project/research'

In [4]:
os.chdir('/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project')
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project'

In [26]:
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Optional, List

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: Dict[str, Any]
    metric_file_name: Path
    target_column: str

    # W&B
    wandb_enabled: bool = True
    wandb_project: str = "customer-churn"
    wandb_entity: Optional[str] = None   # leave None to use default
    wandb_job_type: str = "evaluation"
    wandb_tags: Optional[List[str]] = None

In [7]:
from src.customerchurn.constants import *
from src.customerchurn.utils.common import read_yaml, create_directories, save_json


In [27]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        cfg = self.config.model_evaluation
        params = self.params.model_evaluation  # or self.params.model_trainer if you prefer

        create_directories([cfg.root_dir])

        # schema.yaml has: target_column: exited
        target_col = self.schema.target_column

        # W&B config comes from config.yaml
        wb = self.config.get("wandb", {})
        
        return ModelEvaluationConfig(
            root_dir=Path(cfg.root_dir),
            test_data_path=Path(cfg.test_data_path),
            model_path=Path(cfg.model_path),
            all_params=dict(params),
            metric_file_name=Path(cfg.metric_file_name),
            target_column=target_col,

            wandb_enabled=bool(wb.get("enabled", True)),
            wandb_project=wb.get("project", "customer-churn"),
            wandb_entity=wb.get("entity"),            # can be None
            wandb_job_type=wb.get("job_type", "evaluation"),
            wandb_tags=wb.get("tags", ["churn", "classification"]),
        )

In [28]:
import json
import joblib
import numpy as np
import pandas as pd

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_score, recall_score, f1_score, accuracy_score,
    confusion_matrix
)

import wandb
from src.customerchurn.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def log_to_wandb(self, metrics: dict, cm: np.ndarray | None = None):
        if not self.config.wandb_enabled:
            return

        run = wandb.init(
            project=self.config.wandb_project,
            entity=self.config.wandb_entity,   # None -> use default (works for you)
            job_type=self.config.wandb_job_type,
            tags=self.config.wandb_tags,
            config=self.config.all_params,
            name="model-evaluation",
            reinit=True,
        )

        wandb.log(metrics)

        if cm is not None:
            # W&B confusion matrix plot wants lists
            run.log({
                "confusion_matrix": wandb.plot.confusion_matrix(
                    probs=None,
                    y_true=metrics["_y_true_list"],
                    preds=metrics["_y_pred_list"],
                    class_names=["No Churn", "Churn"]
                )
            })

        # Log artifacts: metrics.json + model
        metrics_path = Path(self.config.metric_file_name)
        if metrics_path.exists():
            art = wandb.Artifact("evaluation-metrics", type="metrics")
            art.add_file(str(metrics_path))
            run.log_artifact(art)

        model_path = Path(self.config.model_path)
        if model_path.exists():
            art = wandb.Artifact("model", type="model")
            art.add_file(str(model_path))
            run.log_artifact(art)

        run.finish()

    def run_evaluation(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column].astype(int)

        # Predictions
        y_pred = model.predict(X_test)

        # Probabilities (if available)
        y_proba = None
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]

        # Metrics
        metrics = {
            "accuracy": float(accuracy_score(y_test, y_pred)),
            "precision": float(precision_score(y_test, y_pred, zero_division=0)),
            "recall": float(recall_score(y_test, y_pred, zero_division=0)),
            "f1": float(f1_score(y_test, y_pred, zero_division=0)),
        }

        if y_proba is not None:
            metrics["roc_auc"] = float(roc_auc_score(y_test, y_proba))
            metrics["pr_auc"] = float(average_precision_score(y_test, y_proba))

        cm = confusion_matrix(y_test, y_pred)

        # Save local metrics.json
        save_json(path=Path(self.config.metric_file_name), data=metrics)

        # W&B plot needs these lists; keep them out of metrics.json
        metrics["_y_true_list"] = y_test.tolist()
        metrics["_y_pred_list"] = y_pred.tolist()

        self.log_to_wandb(metrics=metrics, cm=cm)

        # remove helper fields before returning
        metrics.pop("_y_true_list", None)
        metrics.pop("_y_pred_list", None)

        return metrics

try:
    config = COnfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception aas e:
    raise e

In [30]:
try:
    cm = ConfigurationManager()
    eval_cfg = cm.get_model_evaluation_config()

    evaluator = ModelEvaluation(config=eval_cfg)
    metrics = evaluator.run_evaluation()

    print("Evaluation metrics:", metrics)

except Exception as e:
    raise e

[2026-02-09 15:19:02,769: INFO: common: YAML file config/config.yaml loaded successfully.]
[2026-02-09 15:19:02,790: INFO: common: YAML file params.yaml loaded successfully.]
[2026-02-09 15:19:02,811: INFO: common: YAML file schema.yaml loaded successfully.]
[2026-02-09 15:19:02,812: INFO: common: Directory created at: artifacts]
[2026-02-09 15:19:02,813: INFO: common: Directory created at: artifacts/model_evaluation]
[2026-02-09 15:19:03,079: INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]


wandb: Currently logged in as: rajusubba-project (rajusubba-project-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


accuracy,▁
f1,▁
pr_auc,▁
precision,▁
recall,▁
roc_auc,▁
accuracy,0.8205
f1,0.31879
pr_auc,0.39379
precision,0.50299
recall,0.23333


Evaluation metrics: {'accuracy': 0.8205, 'precision': 0.5029940119760479, 'recall': 0.23333333333333334, 'f1': 0.3187855787476281, 'roc_auc': 0.7445206639566395, 'pr_auc': 0.3937935950674988}
